In [2]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from Adafruit_IO import Client
import time
import threading
base = BaseOverlay("base.bit")
base.is_loaded()
# aio = Client('vsaah', ) #key redacted

True

## Server functions for Brightness module

In [3]:
def setBrightnessRange(low,high,lbtFeed,hbtFeed):
    if lbt < 0 or lbt > 4095:
        print("Error: Low brightness threshold out of range")
        return
    elif hbt < 0 or hbt > 4095:
        print("Error: High brightness threshold out of range")
        return
    elif lbt >= hbt:
        print("Error: Low brightness threshold must be less than High brightness threshold")
        return
    else:
        aio.send(lbtFeed.key, low)
        aio.send(hbtFeed.key, high)

## Helper Threads and Functions for Client

In [4]:
def client_button():
    global stop, sense_brightness
    while not stop:
        if btns.read() == 0x01:
            sense_brightness = False
            stop = True
    print("Button was pressed! Ending program...")
    
def client_LED(LEDStates, LEDLock):
    #Process to handle blinking the LEDs to show various statuses on the client board
    blinkrate = 1
    t_on = 1/(2*blinkrate)
    
    blinkOn = False; #want lights to blink in sync, this stores state
    
    while not stop:
        usingLock = LEDLock.acquire(True)
        if usingLock:
            for i in range(4):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.leds[i].off()
                    else:
                        base.leds[i].on()

            for i in range(4,6):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.rgbleds[i].off()
                    else:
                        base.rgbleds[i].on(0b100)
            LEDLock.release()
        if blinkOn:
            blinkOn = False
        else:
            blinkOn = True
                 
        time.sleep(t_on)
    
def start_blink(LEDStates,index):
    LEDStates[index] = 1
    
def stop_blink(LEDStates,index):
    LEDStates[index] = 0
    led_off(index)
    
def led_off(index):
    if index < 0 or index > 6:
        return
    elif index < 4:
        base.leds[index].off()
    else:
        base.rgbleds[index].off()
        
def led_on(index):
    if index < 0 or index > 6:
        return
    elif index < 4:
        base.leds[index].on()
    else:
        base.rgbleds[index].on(0b100)
        


In [ ]:
led_on(5)

In [ ]:
led_off(5)

## Light Sensor Process and functions

In [5]:
def light_sense(device,LEDStates,LEDLock):
    buf = bytearray(2)
    buf[0] = int('00011000', 2)
    device.write(0x28, buf, 1)

    lbtFeed = aio.feeds('low-brightness-threshold')
    hbtFeed = aio.feeds('high-brightness-threshold')
    bFeed = aio.feeds('brightness')

    sense_brightness = True
    samplectr = 0
    while sense_brightness and not stop:
        if samplectr == 0:
            [lowB, highB] = updateBrightnessThresholds(lbtFeed,hbtFeed)
        samplectr = (samplectr + 1) % SAMPLES_BEFORE_THRESHOLD_UPDATE
        device.read(0x28, buf, 2)
        curr_brightness = buf[0] << 8 | buf[1]
        print("Brightness: {}".format(curr_brightness))
        if curr_brightness < lowB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is lower than threshold!")
        elif curr_brightness > highB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is higher than threshold!")
        else:
            usinglock = LEDLock.acquire(True)
            if usinglock:
                stop_blink(LEDStates,5)
                LEDLock.release()
            
        aio.send(bFeed.key, curr_brightness)
        time.sleep(SAMPLE_RATE)
        
def updateBrightnessThresholds(lbtFeed,hbtFeed):
    lowB = int(aio.receive(lbtFeed.key).value)
    highB = int(aio.receive(hbtFeed.key).value)
    print("Brightness threshold updated: {} to {}".format(lowB,highB))
    return [lowB,highB]

In [ ]:
SAMPLE_RATE = 2 #number of seconds to wait between each sample
SAMPLES_BEFORE_THRESHOLD_UPDATE = 5 #number of samples to take before updating threshold again

stop = False
LEDStates = [0, 0, 0, 0, 0, 0] #store states of LEDs. 0 = steady, 1 = blinking

LEDLock = threading.Lock()

btns = base.btns_gpio
#open and configure Pmod AD2
liba = MicroblazeLibrary(base.PMODB, ['i2c'])
device = liba.i2c_open(3,2)

t1 = threading.Thread(target=light_sense, args=(device,LEDStates,LEDLock))
t1.start()
t2 = threading.Thread(target=client_LED, args=(LEDStates,LEDLock)) 
t2.start()
t3 = threading.Thread(target=client_button, args=())
t3.start()
t1.join()
print('t1 joined')
t2.join()
print('t2 joined')
t3.join()
print('t3 joined')
print('Done')


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-83f808fc2805>", line 14, in light_sense
NameError: name 'lbtFeed' is not defined


t1 joined
